In [1]:
import PIL
from keras import models
from keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import os
from keras import callbacks
import pandas as pd

Using TensorFlow backend.


In [2]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
from efficientnet.keras import EfficientNetB0 as Net
from efficientnet.keras import center_crop_and_resize, preprocess_input
conv_base = Net(weights='imagenet')
height = width = conv_base.input_shape[1]
input_shape = (height, width, 3)

In [4]:
dataframe = pd.read_csv( '/media/tohn/SSD/shopee/new_train.csv')

batch_size = 20
epochs = 40


base_dir = '/media/tohn/SSD/shopee/'
os.chdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'val')

In [5]:
dataframe

,Unnamed: 0,filename,category,group,filename_mod,category_mod
0,52905,d415f1f957c8f5d541b6067e3ee35141.jpg,0,1,00/d415f1f957c8f5d541b6067e3ee35141.jpg,C0
1,52906,9519ce5c45863b4df9ccaac21f05d34c.jpg,0,2,00/9519ce5c45863b4df9ccaac21f05d34c.jpg,C0
2,52907,21cb9c6d6b20b064fdbede2f6428e35f.jpg,0,3,00/21cb9c6d6b20b064fdbede2f6428e35f.jpg,C0
3,52908,dfca61bee5d02624e79ec70e7e33d316.jpg,0,4,00/dfca61bee5d02624e79ec70e7e33d316.jpg,C0
4,52909,7c0bdae7b1222a4b6c7fdcab8a296b2e.jpg,0,5,00/7c0bdae7b1222a4b6c7fdcab8a296b2e.jpg,C0
...,...,...,...,...,...,...
102725,94704,18fc8700b59fb0d620b36f51e3372193.jpg,40,1,40/18fc8700b59fb0d620b36f51e3372193.jpg,C40
102726,94705,89109f99e9c584362253bbdfbd8cb8e8.jpg,40,2,40/89109f99e9c584362253bbdfbd8cb8e8.jpg,C40
102727,94706,cea517d3f82c1e0ab5147b42541774c8.jpg,40,3,40/cea517d3f82c1e0ab5147b42541774c8.jpg,C40
102728,94707,e8b0dc1b53312ae1c011c01bed67c909.jpg,40,4,40/e8b0dc1b53312ae1c011c01bed67c909.jpg,C40


In [6]:
# loading pretrained conv base model
conv_base = Net(weights='imagenet', include_top=False, input_shape=input_shape)

# create new model with a new classification layer
x = conv_base.output  
global_average_layer = layers.GlobalAveragePooling2D()(x)
dropout_layer_1 = layers.Dropout(0.50)(global_average_layer)
prediction_layer = layers.Dense(max(list(dataframe['category']))+1, activation='softmax')(dropout_layer_1)

model = models.Model(inputs= conv_base.input, outputs=prediction_layer) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [7]:
#showing before&after freezing
print('This is the number of trainable layers '
      'before freezing the conv base:', len(model.trainable_weights))
#conv_base.trainable = False  # freeze เพื่อรักษา convolutional base's weight
for layer in conv_base.layers:
    layer.trainable = False
print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))  #freez แล้วจะเหลือ max pool and dense

This is the number of trainable layers before freezing the conv base: 213
This is the number of trainable layers after freezing the conv base: 2


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# import preprocess_crop
# from keras.applications.inception_v3 import preprocess_input

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe[dataframe['group'] != 1],
        directory = train_dir,
        x_col = 'filename_mod',
        y_col = 'category_mod',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

Found 92458 validated image filenames belonging to 41 classes.


In [10]:
test_generator = test_datagen.flow_from_dataframe(
        dataframe = dataframe[dataframe['group'] == 1],
        directory = train_dir,
        x_col = 'filename_mod',
        y_col = 'category_mod',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

Found 10272 validated image filenames belonging to 41 classes.


In [ ]:
root_logdir = '/media/tohn/SSD/COVID/my_logs'
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir,run_id)
run_logdir = get_run_logdir()

tensorboard_cb = callbacks.TensorBoard(log_dir = run_logdir)

In [ ]:
os.makedirs("./models", exist_ok=True)

def avoid_error(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass

#Training
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      avoid_error(train_generator),
      steps_per_epoch= len(dataframe[dataframe['group'] != 1])//batch_size,
      epochs=epochs,
      validation_data=avoid_error(test_generator), 
      validation_steps= len(dataframe[dataframe['group'] == 1]) //batch_size,
      use_multiprocessing=True, 
      workers=15,
      max_queue_size=80,
      callbacks = [tensorboard_cb])

# model.save('./models/B7_R1.h5')

In [ ]:
#Unfreez
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block7a_se_excite':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
print('This is the number of trainable layers '
      'after freezing the conv base:', len(model.trainable_weights))  

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

run_logdir = get_run_logdir()

tensorboard_cb = callbacks.TensorBoard(run_logdir)
#early_stop_cb = callbacks.EarlyStopping(monitor='val_acc', patience=66, mode= 'max')

history = model.fit_generator(
      avoid_error(train_generator),
      steps_per_epoch= len(dataframe[dataframe['group'] != 1])//batch_size,
      epochs=epochs,
      validation_data=avoid_error(test_generator), 
      validation_steps= len(dataframe[dataframe['group'] == 1]) //batch_size,
      use_multiprocessing=True, 
      workers=15,
      max_queue_size=80,
      callbacks = [tensorboard_cb,callbacks.ModelCheckpoint(filepath='./models/checkpoint.h5', save_freq = 'epoch')])

# model.save('./models/B7_R2.h5')